In [ ]:
'''
모두의 연구소 강화학습 이론
http://www.modulabs.co.kr/RL_library/3305

AC코드

https://towardsdatascience.com/reinforcement-learning-w-keras-openai-actor-critic-models-f084612cfd69


AC pytorch
https://github.com/dgriff777/rl_a3c_pytorch


AC tensorflow
https://github.com/hunkim/ReinforcementZeroToAll/blob/master/10_1_Actor_Critic.ipynb

A3C이론

https://www.slideshare.net/WoongwonLee/rlcode-a3c

레이싱 AC

https://github.com/yanpanlau/DDPG-Keras-Torcs/blob/master/ActorNetwork.py
https://yanpanlau.github.io/2016/10/11/Torcs-Keras.html

actor-critic model 만들기

https://github.com/rlcode/reinforcement-learning-kr/blob/master/2-cartpole/2-actor-critic/cartpole_a2c.py
'''

In [4]:
import numpy as np 
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import Add, Multiply
from keras.optimizers import Adam
import keras.backend as K
import tensorflow as tf
import random
from collections import deque

In [6]:
input_layer = Input(shape = [4])
h0 = Dense(16, activation = 'relu')(input_layer)
h1 = Dense(16, activation = 'relu')(h0)
result = Dense(1,activation = 'sigmoid')(h1)
model = Model(inputs = input_layer,outputs=result)


In [16]:
model.output

<tf.Tensor 'dense_6/Sigmoid:0' shape=(?, 1) dtype=float32>

In [26]:
model.trainable_weights

[<tf.Variable 'dense_4/kernel:0' shape=(4, 16) dtype=float32_ref>,
 <tf.Variable 'dense_4/bias:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'dense_5/kernel:0' shape=(16, 16) dtype=float32_ref>,
 <tf.Variable 'dense_5/bias:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'dense_6/kernel:0' shape=(16, 1) dtype=float32_ref>,
 <tf.Variable 'dense_6/bias:0' shape=(1,) dtype=float32_ref>]

In [22]:
action_gradient = tf.placeholder(tf.float32,[None,4])

<tf.Tensor 'Placeholder_2:0' shape=(?, 4) dtype=float32>

In [21]:
tf.gradients(model.output,model.trainable_weights, action_gradient)

[<tf.Tensor 'gradients/dense_4/MatMul_grad/MatMul_1:0' shape=(4, 16) dtype=float32>,
 <tf.Tensor 'gradients/dense_4/BiasAdd_grad/BiasAddGrad:0' shape=(16,) dtype=float32>,
 <tf.Tensor 'gradients/dense_5/MatMul_grad/MatMul_1:0' shape=(16, 16) dtype=float32>,
 <tf.Tensor 'gradients/dense_5/BiasAdd_grad/BiasAddGrad:0' shape=(16,) dtype=float32>,
 <tf.Tensor 'gradients/dense_6/MatMul_grad/MatMul_1:0' shape=(16, 1) dtype=float32>,
 <tf.Tensor 'gradients/dense_6/BiasAdd_grad/BiasAddGrad:0' shape=(1,) dtype=float32>]

In [23]:
tf.gradients(model.output,model.trainable_weights, -action_gradient)

[<tf.Tensor 'gradients_1/dense_4/MatMul_grad/MatMul_1:0' shape=(4, 16) dtype=float32>,
 <tf.Tensor 'gradients_1/dense_4/BiasAdd_grad/BiasAddGrad:0' shape=(16,) dtype=float32>,
 <tf.Tensor 'gradients_1/dense_5/MatMul_grad/MatMul_1:0' shape=(16, 16) dtype=float32>,
 <tf.Tensor 'gradients_1/dense_5/BiasAdd_grad/BiasAddGrad:0' shape=(16,) dtype=float32>,
 <tf.Tensor 'gradients_1/dense_6/MatMul_grad/MatMul_1:0' shape=(16, 1) dtype=float32>,
 <tf.Tensor 'gradients_1/dense_6/BiasAdd_grad/BiasAddGrad:0' shape=(1,) dtype=float32>]

In [ ]:
x = tf.constant(1.0)
b = tf.constant(2.0)
y = x + b

In [ ]:
class A3C_model:
    def __init__(self):
        self.observation_space = 0
        self.action_space = 0

        self.learning_rate = 0.001
        self.epsilon = 1.0
        self.epsilon_decay = .995
        self.gamma = .95
        self.tau   = .125
        self.memory = deque(maxlen=2000)

        self.actor_state_input, self.actor_model = self.create_actor_model()
        _, self.target_actor_model = self.create_actor_model()

        self.actor_critic_grad = tf.placeholder(tf.float32, \ 
                [None, self.env.action_space.shape[0]]) 

        actor_model_weights = self.actor_model.trainable_weights

        self.actor_grads = tf.gradients(self.actor_model.output, \
                actor_model_weights, -self.actor_critic_grad)

        grads = zip(self.actor_grads, actor_model_weights)

        self.optimize =  tf.train.AdamOptimizer(\
                self.learning_rate).apply_gradients(grads)
    def create_actor_model(setlf):
        state_input = Input(shape=self.observation_space)
        h1 = Dense(24, activation='relu')(state_input)
        h2 = Dense(48, activation='relu')(h1)
        h3 = Dense(24, activation='relu')(h2)
        output = Dense(self.action_space,  
            activation='relu')(h3)
        
        model = Model(input=state_input, output=output)
        adam  = Adam(lr=0.001)
        model.compile(loss="mse", optimizer=adam)
        return state_input, model
    
    